In [42]:
from _linear import LinearRegression
import _dgp as dg
from _metrics import mse_score, CrossValidation, accuracy_score,f1_score
from  _logistic import LogisticRegression
import numpy as np
import pandas as pd

## **Table of Contents**

- [Linear Regression](#linear-regression)
  - [Generate data and fit first model](#generate-data-and-fit-first-model)
  - [Evaluate model performance](#evaluate-model-performance)
  - [Automatic selection of variables](#automatic-selection-of-variables) 
  - [Drawbacks and to do list](#drawbacks-and-to-do-list)

- [Logistic Regression](#logistic-regression)
  - [Work with excel data (5 variables in total)](#work-with-excel-data-5-variables-in-total)
  - [Evaluate model performance](#evaluate-model-performance-logistic-regression)
  - [Automatic selection of variables](#automatic-selection-of-variables-logistic-regression)
  - [Generate 50 variables and autoselection](#Generate-50-variables-and-autoselection)



- [Logistic Regression OVR](#Logistic-Regression-OVR)
  - [Fit and predict model](#Fit-and-predict-model)
  - [Automatic selection of variables](#automatic-selection-of-variables-Log-OVR)
  - [Evaluate model performance](#Evaluate-model-performance-Log-OVR)



- [Logistic Regression Softmax](#Logistic-Regression-Softmax)
  - [Fit and predict model](#Fit-and-predict-model-Log-SFTMX)
  - [Evaluate model performance](#Evaluate-model-performance-Log-SFTMX)
  - [Automatic selection of variables](#automatic-selection-of-variables-Log-SFTMX)



## **Linear Regression** <a id='linear-regression'></a>


### **Generate data and fit first model**

Generate data with no correlation with only 5 true variables among 50 variables

In [43]:
x,y=dg.gen(type=1)
#get matrix size to understand what is the data begind
print(x.shape)
print(y.shape)

(500, 50)
(500,)


Run linear regression and get B parameter

In [44]:
#use Newton Raphson optimisation which is the best optimiser for parametric models
regression=LinearRegression(solver="nr")
regression.fit(x,y)

algorithm did  converge under 100 iterations (at 2 iterations)


array([-2.08826857e-03,  2.50707569e+00,  1.30029974e+00,  4.94942745e-01,
       -5.05120915e-01, -3.40735850e+00,  6.38589597e-03, -8.21298506e-04,
       -1.70335093e-03, -3.79786833e-03, -7.18114798e-04, -9.51195111e-03,
        2.13622281e-03,  5.08380226e-03,  2.54999630e-04, -7.12511603e-03,
       -5.04421433e-03, -3.77307133e-03, -8.96545318e-03, -4.26558726e-03,
        1.65588231e-03,  3.53276168e-03, -3.24868487e-03,  4.38376497e-04,
       -2.08058274e-03,  6.80588827e-05, -2.22418925e-03,  7.64943744e-03,
        2.12951747e-03,  2.14148214e-03,  4.50778398e-03, -7.98374734e-03,
        4.06051251e-03, -3.60530350e-04,  7.67713195e-03, -1.08796243e-03,
        4.20013661e-03, -5.62492094e-03, -2.54044950e-03, -2.40866424e-03,
       -4.68046528e-03, -3.31485836e-03, -3.63670848e-03,  4.05998926e-03,
       -9.58988274e-03, -3.16537090e-03,  4.15397112e-03, -6.37431764e-03,
       -2.75352976e-03, -4.23764453e-03, -9.75608027e-03])

As we used NR optimisation, it converged very fast, only in 2 iterations

if We want to interpret Linear regression results, we can do so by typing this command:

In [21]:
regression.get_inference().head()

,params,std,t value,p value
0,-0.005344,0.004268,-1.252140,0.2112
1,2.498205,0.004386,569.563428,0.0000
2,1.303919,0.004263,305.873759,0.0000
3,0.499912,0.004158,120.227360,0.0000
4,-0.498036,0.004299,-115.858478,0.0000


### **Evaluate model performance**

We can evaluate model performance by looking at Mean squared error

In [16]:
mse_score(y,regression.predict(x))

0.008947520780878555

However, the best way to evaluate model performance remains Cross Validation for several reasons

In [17]:
list_of_mse=CrossValidation(Class_algorithm=regression,x=x,y=y,metrics_function=mse_score,nb_k_fold=6)
print(list_of_mse)

algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
algorithm did  converge under 100 iterations (at 2 iterations)
[0.009339829513632726, 0.010377676180038776, 0.012065743000578374, 0.013332362334853482, 0.013774820256179649, 0.009398173839300278, 0.005305990637966489]


we can average these results to get average model performance

In [9]:
np.mean(list_of_mse)

0.01009195375055408

### **Automatic selection of variables**

**However, it is not a good practice to perform linear regression using all variables.** In order to obtain the best subset, several strategies can be adopted:

- **VIF models:** These identify and drop collinear variables to avoid multicollinearity issues.
- **Lasso regression:** This technique can be used for feature selection by penalizing the absolute size of the regression coefficients.
- **Forward/Backward/Stepwise selection:** These are iterative methods for feature selection, where variables are added or removed based on their impact on model performance.

In this section we will use last suggestion


In [22]:
col_index=regression.autoselection("forward","BIC_ll",print_message=False)
col_index

array([4, 0, 1, 2, 3])

and we get the following columns of X Matrix , first 5 variables that are also 5 true variables that we defined in dgp

In [23]:
x[:,col_index]

array([[ 0.01516751,  0.04356754,  1.22018023,  1.16884148, -0.18863908],
       [ 0.38470383, -0.50790125, -0.6858267 , -1.07900273,  0.6559498 ],
       [-1.26799987,  1.80332023,  0.35896385,  0.33896862, -0.38083356],
       ...,
       [-0.02579775,  1.27848172,  0.34700335,  0.97544884, -1.51427759],
       [-0.46805553, -1.06037772, -1.44214529,  0.79863867,  0.39274082],
       [ 0.71841844, -0.82243412,  0.43663251,  0.71719104, -1.30066039]])

In [24]:
col_index=regression.autoselection("stepwise","BIC_ll",print_message=False)
col_index

array([4, 0, 1, 2, 3])

In [28]:
col_index=regression.autoselection("backward","BIC_ll",print_message=False)
col_index

array([0, 1, 2, 3, 4])

### **Drawbacks and to do list**

**In this section, we discussed the machine learning approach of estimating the model parameters of Linear Regression.**

<span style="color:red"> However, we did not analyze the validity tests of normality and homoscedasticity of residuals, endogenous variables, etc. This section will be developed soon.</span>









## **Logistic Regression**

### **Work with excel data (5 variables in total)**

Load data and explore 

In [45]:
table=pd.read_csv("data/exampleLR.csv")

In [3]:
table.head()

,Unnamed: 0,V1,V2,V3,V4,V5,y
0,0,-0.560476,-0.601893,-0.995799,-0.820987,-0.511604,0
1,1,-0.230177,-0.993699,-1.039955,-0.307257,0.236938,0
2,2,1.558708,1.026785,-0.017980,-0.902098,-0.541589,1
3,3,0.070508,0.751061,-0.132175,0.627069,1.219228,0
4,4,0.129288,-1.509167,-2.549343,1.120355,0.174136,0


Always convert data to numpy array before starting the work

In [46]:
x,y=np.array(table.drop(["y","Unnamed: 0"],axis=1)),np.array(table["y"])
column_names_x=table.drop(["y","Unnamed: 0"],axis=1).columns.tolist()

**Are classes balanced?**

- If yes, the use of accuracy is enough (our case).
- If not:
  - Use other metrics such as precision, recall, F1 score, etc.
  - Do over/undersampling.



In [4]:
table["y"].value_counts()

y
0    253
1    247
Name: count, dtype: int64

Try Gradient Descent algorithm

In [47]:
model=LogisticRegression(solver="gd",add_intercept=False)
model.fit(x,y)

algorithm did not converge under 100 iterations,so the calculated parameter is biased


array([-0.98177656,  1.93284507,  0.89477738,  0.77241981, -1.06475169])

**Algorithm could not converge:**
If you have this message, **NEVER** proceed to further steps as 
the estimated parameter is biased. 

Therefore, try other hyperparameters or optimisations to aim  convergence.


In [48]:
model1=LogisticRegression(solver="gd",learning_rate=0.004,max_iteration=300,add_intercept=True)
model1.fit(x,y)

algorithm did  converge under 300 iterations (at 79 iterations)


array([-0.14211795, -1.10847242,  2.18459166,  1.02655283,  0.87139786,
       -1.20943036])

We can also try NR optimisation that will converge much more faster

Be very cautious with this method as it requires that X matrix is of full rank which is not guaranteed in case of image or text classification

In [30]:
model2=LogisticRegression(solver="nr")
model2.fit(x,y)

algorithm did  converge under 100 iterations (at 6 iterations)


array([-0.14299915, -1.11517033,  2.19757232,  1.03328488,  0.87638437,
       -1.21658404])

Try stochastic gradient descent with 68 as size of mini batch to see the performance

In [31]:
model3=LogisticRegression(solver="sgd",learning_rate=0.001,max_iteration=300,mini_batch_size=68,add_intercept=False)
model3.fit(x,y)

algorithm did  converge under 2400 iterations (at 1264 iterations)


array([-1.05436949,  2.0732936 ,  0.96684633,  0.82591372, -1.14325812])

With new learning rate algorithm converged so we will use results of model1 to make predictions

In [49]:
#predict labels 
model1.predict(x,last_class=False)[0:5]


array([0., 0., 1., 1., 0.])

In [50]:
#get probabilities of ALL classes 
model1.proba[0:5]

array([[0.12404903, 0.87595097],
       [0.02461034, 0.97538966],
       [0.55568638, 0.44431362],
       [0.58823128, 0.41176872],
       [0.00434789, 0.99565211]])

We can access Information Criteria of the model

In [51]:
model1.get_inference(only_IC=True)

{'LL': -177.40450034974424,
 'AIC_ll': 366.8090006994885,
 'BIC_ll': 392.0966492900216}

As well as conclude about parameter significance

In [52]:
model1.get_inference(only_IC=False,ordered_columns_names=column_names_x)

,coef,std,z value,p value,p value star,odds ratio
in_1,-0.1421,0.13,-1.07,0.286,,0.87
V1_1,-1.1085,0.16,-6.96,0.0,***,0.33
V2_1,2.1846,0.22,9.81,0.0,***,8.89
V3_1,1.0266,0.16,6.27,0.0,***,2.79
V4_1,0.8714,0.14,6.03,0.0,***,2.39
V5_1,-1.2094,0.17,-6.98,0.0,***,0.3


### **Evaluate model performance** (Logistic Regression)

Evaluate model with accuracy and f1score 

In [11]:
#one shot accuracy score
accuracy_score(y,model1.predict(x,last_class=False))

0.844

In [12]:
#using cross validation
CrossValidation(Class_algorithm=model1,x=x,y=y,metrics_function=accuracy_score,nb_k_fold=4)

algorithm did  converge under 300 iterations (at 87 iterations)
algorithm did  converge under 300 iterations (at 110 iterations)
algorithm did  converge under 300 iterations (at 117 iterations)
algorithm did  converge under 300 iterations (at 104 iterations)


[0.92, 0.824, 0.8, 0.824]

In [13]:
CrossValidation(Class_algorithm=model1,x=x,y=y,metrics_function=f1_score,nb_k_fold=4)

algorithm did  converge under 300 iterations (at 89 iterations)
algorithm did  converge under 300 iterations (at 100 iterations)
algorithm did  converge under 300 iterations (at 97 iterations)
algorithm did  converge under 300 iterations (at 119 iterations)


[0.8596491228070176, 0.8702290076335878, 0.8, 0.7804878048780488]

### **Automatic selection of variables** (Logistic Regression)

In [14]:
index_cols=model1.autoselection("forward","BIC_ll",print_message=False)
index_cols

array([1, 4, 0, 2, 3])

In [15]:
index_cols=model1.autoselection("backward","BIC_ll",print_message=False)
index_cols

array([0, 1, 2, 3, 4])

In [16]:
index_cols=model1.autoselection("stepwise","BIC_ll",print_message=False)
index_cols

array([4, 1, 0, 2, 3])

As you see in this case we get 5 variables which are 5 true variables

We can see how this selection performs on another Data Generation Process that will contain 5 true variables and 45 false variables

### **Generate 50 variables and autoselection**

In [2]:
x,y=dg.gen(1,"logistic")

In [28]:
model_new=LogisticRegression(solver="nr")
model_new.fit(x,y)

algorithm did  converge under 100 iterations (at 9 iterations)


array([ 3.68356259e-01,  4.17576652e+00,  2.45656155e+00,  1.14517428e+00,
       -9.25465015e-01, -6.14405346e+00, -1.57876464e-01, -3.98723522e-01,
        2.89763066e-01,  4.07088701e-01,  2.68217231e-01,  5.37650899e-01,
        5.51471596e-01,  2.77787653e-02, -1.23918994e-01,  3.01416615e-02,
        2.23956451e-03, -5.17056366e-01, -2.88207203e-01,  2.20257943e-01,
       -1.19508841e-01, -1.58358294e-01, -3.31212802e-01, -9.96478299e-02,
       -9.15638576e-02,  2.45833086e-01, -8.03545080e-02, -3.24741110e-01,
        2.09788239e-01, -2.67504329e-01, -1.91994199e-02, -1.95555148e-01,
       -1.94559088e-02, -2.21139792e-02, -2.78842802e-01,  4.98628510e-01,
        2.09473275e-01, -1.23428219e-01,  3.06597942e-01,  4.09820588e-01,
        2.43880795e-02,  1.22607848e-01,  4.75156044e-01,  2.22176634e-01,
       -1.35562492e-01, -4.94108726e-01, -3.46332998e-01, -7.30170182e-02,
        9.33952903e-02,  2.61361158e-01,  4.24655606e-02])

In [29]:
model_new.autoselection("forward","BIC_ll",print_message=False)

array([4, 0, 1, 3, 2])

In [55]:
model_new.autoselection("stepwise","BIC_ll",print_message=False)

array([3, 4, 0, 1, 2])

In [14]:
model_new.autoselection("backward","BIC_ll",print_message=False)

array([0, 1, 2, 3, 4])

Despite having 50 variables in the dataset, the model successfully identified only 5 true variables. All 3 models gave same results which is a normal behaviour.

**Let's now generalise this approach for multiclass classification using OVR algoritmn and then softmax regression**

## **Logistic Regression OVR**



Logistic OVR is a multiclass classification strategy where you train multiple binary classifiers, each distinguishing between one class and the rest.

**Classifier Training**:
   - For each class:
     - Transform data into binary classification (class vs. rest).
     - Train binary classifier (e.g., logistic regression).

**Classification**:
   - Predict class by selecting the classifier with highest probability.



### **Fit and predict model**

Load data

In [17]:
table=pd.read_csv("data/exampleLRMulti.csv")

Get rid of unnamed columns

In [18]:
table.drop(table.columns[table.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

One more time classes are distributed equally, so we can estimate model performance with accuracy score

In [19]:
table["column_Y"].value_counts()

column_Y
1    336
0    333
2    331
Name: count, dtype: int64

In [20]:
x,y=np.array(table.drop(["column_Y"],axis=1)),np.array(table["column_Y"])

Estimate Multiclass Logistic Regression with One vs Rest algorithm and Newton Raphson optimisation 

Since it's an OVR algorithm, it will run model 3 times with 3 columns of estimated parameter

In [21]:
modelOVR=LogisticRegression(solver="nr",multiclass="ovr",add_intercept=False)
modelOVR.fit(x,y)

algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)


array([[ 0.12241142,  0.2961772 , -0.84392429],
       [ 0.0618641 , -0.77927438,  0.39345262],
       [ 0.0615473 ,  0.00813252, -0.01878925],
       [ 0.02809831,  0.07180074, -0.20671125],
       [-0.56575934,  0.47820965, -0.49276421],
       [ 0.02019647, -0.10160388,  0.05084678],
       [-0.49125782,  0.24025151,  0.44457739],
       [-0.0106063 ,  0.15383126, -0.06228379],
       [ 0.77078083, -0.09937053, -0.05505139],
       [ 0.01491474, -0.01253306,  0.05004675]])

We can also predict. Predictions are made always in the ascending order: 

    -first column will be 0 class
    -second column will be 1 class
    -third column will be 2 class


In [22]:
predictionsOVR=modelOVR.predict(x)
predictionsOVR 

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]])

We will compare these predictions to true one hot encoded labels

In [23]:
modelOVR.y

array([[0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1]])

Of course, we can always get probability (for OVR it is not normalised, but for predicted values it is normalised!!)

In [24]:
modelOVR.proba

array([[0.01308505, 0.9858762 , 0.00103875],
       [0.29274103, 0.5361041 , 0.17115487],
       [0.0418237 , 0.34143318, 0.61674312],
       ...,
       [0.01818236, 0.12386295, 0.85795469],
       [0.31591108, 0.46589593, 0.21819298],
       [0.02561   , 0.27372462, 0.70066538]])

get information criteria

In [25]:
modelOVR.get_inference(only_IC=True)

{'LL': -783.7514402342946,
 'AIC_ll': 1587.5028804685892,
 'BIC_ll': 1636.5804332584105}

### **Evaluate model performance** (Log OVR)

We can evaluate accuracy

In [27]:
accuracy_score(predictionsOVR, modelOVR.y)

0.694

But we want more intelligent evaluation, so we use cross validation 

In [28]:
acc_list=CrossValidation(Class_algorithm=modelOVR,x=x,y=modelOVR.y,metrics_function=accuracy_score,nb_k_fold=4)

algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 4 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)
algorithm did  converge under 100 iterations (at 5 iterations)


By averaging we get more "robust" accuracy 

In [29]:
print(acc_list)
np.mean(acc_list)

[0.664, 0.708, 0.66, 0.648]


0.67

### **Automatic selection of variables** (Log OVR)

We can also do variable selection as we did before, but in this case we will use generalised positive cross entropy as Log Likelihood 

In [30]:
modelOVR.autoselection("forward","BIC_ll",print_message=False)


array([0, 1, 8, 6, 4])

In [99]:
modelOVR.autoselection("backward","BIC_ll",print_message=False)


array([0, 1, 4, 6, 8])

In [100]:
col_index=modelOVR.autoselection("stepwise","BIC_ll",print_message=False)
col_index

array([6, 1, 8, 0, 4])

## **Logistic Regression Softmax**

**Now, let's see another way of solving multicase without running P independent classifiers-> logistic softmax**

### **Fit and predict model** (Log SFTMX)

In [31]:
from _linear import LinearRegression
import _dgp as dg
from _metrics import mse_score, CrossValidation, accuracy_score,f1_score
from  _logistic import LogisticRegression
import numpy as np
import pandas as pd

In [32]:
table=pd.read_csv("data/exampleLRMulti.csv")
table.drop(table.columns[table.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
x,y=np.array(table.drop(["column_Y"],axis=1)),np.array(table["column_Y"])
columns_x=table.drop(["column_Y"],axis=1).columns.tolist()

In [33]:
modelSFT=LogisticRegression(solver="gd",add_intercept=True)#dont use nr for the moment
modelSFT.fit(x,y)

algorithm did  converge under 100 iterations (at 85 iterations)


array([[-0.1730108 , -0.49281646],
       [ 0.8199887 ,  0.93342982],
       [-0.27379707, -0.81083862],
       [ 0.0607405 , -0.00339066],
       [ 0.1586324 ,  0.20834796],
       [ 0.01743636,  0.8561222 ],
       [-0.03942523, -0.13332718],
       [-0.72704995, -0.04058517],
       [ 0.04325651,  0.18452295],
       [ 0.64192215, -0.17627129],
       [-0.03731455, -0.06261774]])

As you see, when we have three classes, we consider one of them as reference ( set up to 0 in exponential function when doing the prediction)

We can compare them to modelOVR parameters

In [34]:
modelOVR.params

array([[ 0.12241142,  0.2961772 , -0.84392429],
       [ 0.0618641 , -0.77927438,  0.39345262],
       [ 0.0615473 ,  0.00813252, -0.01878925],
       [ 0.02809831,  0.07180074, -0.20671125],
       [-0.56575934,  0.47820965, -0.49276421],
       [ 0.02019647, -0.10160388,  0.05084678],
       [-0.49125782,  0.24025151,  0.44457739],
       [-0.0106063 ,  0.15383126, -0.06228379],
       [ 0.77078083, -0.09937053, -0.05505139],
       [ 0.01491474, -0.01253306,  0.05004675]])

also predict

In [35]:
modelSFT.predict(x)

array([[0., 1.],
       [0., 1.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 1.],
       [0., 0.]])

In [36]:
modelSFT.proba

array([[0.05422397, 0.93921357, 0.00656247],
       [0.31764523, 0.44164768, 0.24070709],
       [0.06065304, 0.32045348, 0.61889347],
       ...,
       [0.0297319 , 0.10861589, 0.86165221],
       [0.32299072, 0.38938157, 0.2876277 ],
       [0.0404247 , 0.2834663 , 0.676109  ]])

In [37]:
modelOVR.proba

array([[0.01308505, 0.9858762 , 0.00103875],
       [0.29274103, 0.5361041 , 0.17115487],
       [0.0418237 , 0.34143318, 0.61674312],
       ...,
       [0.01818236, 0.12386295, 0.85795469],
       [0.31591108, 0.46589593, 0.21819298],
       [0.02561   , 0.27372462, 0.70066538]])

In [38]:
modelSFT.get_inference(only_IC=True)

{'LL': -487.9286245107654,
 'AIC_ll': 997.8572490215308,
 'BIC_ll': 1051.8425570903344}

In [39]:
modelSFT.get_inference(only_IC=False,ordered_columns_names=columns_x)

,coef,std,z value,p value,p value star,odds ratio
intercep_0,-0.173,0.14,-1.27,0.206,,0.84
column_0_0,0.82,0.08,10.75,0.0,***,2.27
column_1_0,-0.2738,0.08,-3.27,0.001,***,0.76
column_2_0,0.0607,0.09,0.65,0.513,,1.06
column_3_0,0.1586,0.1,1.65,0.098,*,1.17
column_4_0,0.0174,0.08,0.22,0.824,,1.02
column_5_0,-0.0394,0.09,-0.43,0.669,,0.96
column_6_0,-0.727,0.07,-10.16,0.0,***,0.48
column_7_0,0.0433,0.09,0.46,0.644,,1.04
column_8_0,0.6419,0.07,8.97,0.0,***,1.9


Since we have 3 classes and the last one ( class 2 ) is used as a reference, we get two vectors of coefficients for class 0 and 1 as well as their significance

### **Automatic selection of variables** (Log SFTMX)

In [40]:
modelSFT.autoselection("forward","BIC_ll",print_message=False)

array([4, 8, 1, 0, 6])

In [44]:
modelSFT.autoselection("stepwise","BIC_ll",print_message=False)

array([6, 0, 8, 4, 1])

In [45]:
modelSFT.autoselection("backward","BIC_ll",print_message=False)

array([0, 1, 4, 6, 8])

### **Evaluate model performance** (Log SFTMX)

In [41]:
meansftmx=CrossValidation(Class_algorithm=modelSFT,x=x,y=modelSFT.y,metrics_function=accuracy_score,nb_k_fold=4)
np.mean(meansftmx)

algorithm did not converge under 100 iterations,so the calculated parameter is biased
algorithm did  converge under 100 iterations (at 97 iterations)
algorithm did not converge under 100 iterations,so the calculated parameter is biased
algorithm did not converge under 100 iterations,so the calculated parameter is biased


0.76